In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline

In [2]:
data = pd.read_csv('nypd_arrest.csv')

In [3]:
data.head()

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude
0,206892169,12/31/2019,907.0,"IMPAIRED DRIVING,DRUG",347.0,INTOXICATED & IMPAIRED DRIVING,VTL11920U4,M,M,33,0,25-44,M,WHITE,1000916,245710,40.841085,-73.939769
1,206888084,12/31/2019,739.0,"FRAUD,UNCLASSIFIED-FELONY",112.0,THEFT-FRAUD,PL 1908301,F,Q,101,0,25-44,M,BLACK HISPANIC,1055868,156522,40.596012,-73.742116
2,206890433,12/31/2019,122.0,"HOMICIDE, NEGLIGENT, VEHICLE,",102.0,HOMICIDE-NEGLIGENT-VEHICLE,PL 1251201,F,M,23,0,18-24,M,BLACK,997462,227024,40.789803,-73.952289
3,206890441,12/31/2019,268.0,CRIMINAL MIS 2 & 3,121.0,CRIMINAL MISCHIEF & RELATED OF,PL 1450502,F,S,120,0,18-24,M,BLACK,962822,174282,40.645023,-74.077217
4,206890973,12/31/2019,101.0,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,M,25,97,25-44,M,BLACK,1004138,226326,40.787875,-73.928182


In [4]:
data.drop(['ARREST_KEY', 'PD_CD', 'KY_CD', 'LAW_CODE', ], axis=1, inplace=True)

In [5]:
data.dropna(inplace=True)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213089 entries, 0 to 214616
Data columns (total 14 columns):
ARREST_DATE          213089 non-null object
PD_DESC              213089 non-null object
OFNS_DESC            213089 non-null object
LAW_CAT_CD           213089 non-null object
ARREST_BORO          213089 non-null object
ARREST_PRECINCT      213089 non-null int64
JURISDICTION_CODE    213089 non-null int64
AGE_GROUP            213089 non-null object
PERP_SEX             213089 non-null object
PERP_RACE            213089 non-null object
X_COORD_CD           213089 non-null int64
Y_COORD_CD           213089 non-null int64
Latitude             213089 non-null float64
Longitude            213089 non-null float64
dtypes: float64(2), int64(4), object(8)
memory usage: 24.4+ MB


In [8]:
data.rename(columns={'PD_DESC':'offenses', 'LAW_CAT_CD':'offense_type'}, inplace=True)

In [7]:
#data.PERP_RACE.value_counts(normalize=True).mul(100).round(2).astype(str) + ' %'

In [9]:
#data.ARREST_BORO.value_counts()

In [10]:
#data.PD_DESC.value_counts()[:10]

In [11]:
#data.ARREST_PRECINCT.value_counts()[:20]

In [13]:
from sklearn.preprocessing import LabelEncoder

In [12]:
X = data[['offenses', 'offense_type', 'ARREST_BORO', 'AGE_GROUP']].values

In [14]:
lab_enc = LabelEncoder()
X[:,1] = lab_enc.fit_transform(X[:,1])
X[:,2] = lab_enc.fit_transform(X[:,2])
X[:,3] = lab_enc.fit_transform(X[:,3])

In [15]:
X[:,0] = lab_enc.fit_transform(X[:,0])

In [16]:
y = lab_enc.fit_transform(data[['PERP_RACE']].values)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X_train_all, X_test, y_train_all, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

In [33]:
X_train, X_val, y_train, y_val = train_test_split( X_train_all, y_train_all, test_size=0.2, random_state=42)

log = LogisticRegression(solver='lbfgs', multi_class='multinomial')
log.fit(X_train, y_train)
y_pred = log.predict(X_train)
print('training accuracy: ', accuracy_score(y_train, y_pred))

training accuracy:  0.47591951663


In [36]:
log_y_pred_val = log.predict(X_val)
print('Logistic Validation accuracy: ', accuracy_score(y_val, log_y_pred_val))

Logistic Validation accuracy:  0.47942513565


In [37]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_train)
print('knn training accuracy: ', accuracy_score(y_train, y_pred_knn))

knn training accuracy:  0.443318472458


In [38]:
y_pred_knn = knn.predict(X_val)
print('knn validaton accuracy: ', accuracy_score(y_val, y_pred_knn))

knn validaton accuracy:  0.435078457252


In [39]:
from sklearn.svm import LinearSVC

svc = LinearSVC(C=10)
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_train)
print('svc training accuracy: ', accuracy_score(y_train, y_pred_svc))

svc training accuracy:  0.477613363055


In [40]:
y_pred_svc_val = svc.predict(X_val)
print('svc validation accuracy: ', accuracy_score(y_val, y_pred_svc_val))

svc validation accuracy:  0.481067605221


In [41]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
y_pred_rfc = rfc.predict(X_train)
print('Random forest training accuracy: ', accuracy_score(y_train, y_pred_rfc))

Random forest training accuracy:  0.505008212589


In [42]:
y_pred_rfc_val = rfc.predict(X_val)
print('Random forest validation accuracy: ', accuracy_score(y_val, y_pred_rfc_val))

Random forest validation accuracy:  0.489925208975
